In [22]:
import numpy
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import torch.utils.data as Data
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor

data=pd.read_excel('C:/Users/XHM/Desktop/data/TFT/TFT_train.xlsx',engine='openpyxl')
all_input=data[['Vdrain','Vgate','W/L']].values
all_output=data['Idrain']
minmax=preprocessing.MinMaxScaler()
#all_output=minmax.fit_transform(data['Idrain'].values.reshape(-1,1))

(x_train,x_test,y_train,y_test)=train_test_split(all_input,all_output,train_size=0.8,
                                                 random_state=0)


#数据类型处理
train_xt=torch.from_numpy(x_train.astype(np.float32))
train_yt=torch.from_numpy(y_train.values)
test_xt=torch.from_numpy(x_test.astype(np.float32))
test_yt=torch.from_numpy(y_test.values)
#将数据处理为数据加载器
train_data=Data.TensorDataset(train_xt,train_yt.float())
test_data=Data.TensorDataset(test_xt,test_yt.float())

train_loader=Data.DataLoader(dataset=train_data,batch_size=64,shuffle=True,num_workers=0)

## 网格搜索

In [23]:
def train():
    mlp = MLPRegressor()
    # 查看默认超参
    print(mlp)
    # 训练模型
    mlp.fit(train_xt, train_yt)
    print(mlp.score(train_xt, train_yt)) 
train()

MLPRegressor()
0.9311243025292835


In [ ]:
from sklearn.neural_network import MLPRegressor
import numpy as np
import pickle
import gzip
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

mlp_clf__tuned_parameters = {"hidden_layer_sizes": [(100,), (100, 30),(100,50,20)],
                             "solver": ['adam', 'sgd', 'lbfgs'],
                             "max_iter": [20],
                             "verbose": [True]
                             }
mlp = MLPRegressor()
estimator = GridSearchCV(mlp, mlp_clf__tuned_parameters, n_jobs=6)
estimator.fit(train_xt, train_yt)

## lazy搜索

In [7]:
from lazypredict.Supervised import LazyRegressor
from sklearn import datasets
from sklearn.utils import shuffle
import numpy as np

reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
models, predictions = reg.fit(x_train,x_test, y_train, y_test)

print(models)

 21%|█████████████████▊                                                                 | 9/42 [00:01<00:03,  8.44it/s]

GammaRegressor model failed to execute
Some value(s) of y are out of the valid range for family GammaDistribution


 69%|████████████████████████████████████████████████████████▌                         | 29/42 [00:08<00:04,  2.76it/s]

PoissonRegressor model failed to execute
Some value(s) of y are out of the valid range for family PoissonDistribution


100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:10<00:00,  4.09it/s]

                               Adjusted R-Squared  R-Squared  RMSE  Time Taken
Model                                                                         
GaussianProcessRegressor                     1.00       1.00  0.01        2.15
ExtraTreesRegressor                          1.00       1.00  0.12        0.41
KNeighborsRegressor                          1.00       1.00  0.46        0.04
RandomForestRegressor                        1.00       1.00  0.52        0.43
BaggingRegressor                             1.00       1.00  0.55        0.07
DecisionTreeRegressor                        1.00       1.00  0.66        0.01
ExtraTreeRegressor                           1.00       1.00  0.73        0.01
HistGradientBoostingRegressor                1.00       1.00  1.32        0.42
XGBRegressor                                 1.00       1.00  1.32        0.15
LGBMRegressor                                1.00       1.00  1.32        0.08
GradientBoostingRegressor                    1.00   

## 贝叶斯应用于随机森林

In [34]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
from sklearn.metrics import r2_score

# 产生随机分类数据集，10个特征， 2个类别
x, y = make_classification(n_samples=1000,n_features=10,n_classes=2)
rf = RandomForestClassifier()
print(np.mean(cross_val_score(rf, x, y, cv=20, scoring='roc_auc')))

0.9930395512820513


In [35]:
def rf_cv(n_estimators, min_samples_split, max_features, max_depth):
    val = cross_val_score(
        RandomForestClassifier(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=min(max_features, 0.999), # float
            max_depth=int(max_depth),
            random_state=2
        ),
        x, y, scoring='roc_auc', cv=5
    ).mean()
    return val

In [11]:
 rf_bo = BayesianOptimization(
        rf_cv,
        {'n_estimators': (10, 250),
        'min_samples_split': (2, 25),
        'max_features': (0.1, 0.999),
        'max_depth': (5, 15)}
    )
rf_bo.maximize()

|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        |  0.9724   |  7.914    |  0.2049   |  7.509    |  89.04    |
|  2        |  0.9715   |  8.792    |  0.4519   |  23.19    |  109.5    |
|  3        |  0.9687   |  5.918    |  0.9053   |  14.88    |  140.2    |
|  4        |  0.9698   |  12.84    |  0.5776   |  6.488    |  109.8    |
|  5        |  0.9662   |  10.52    |  0.9554   |  5.215    |  209.6    |
|  6        |  0.9723   |  7.675    |  0.297    |  7.048    |  90.36    |
|  7        |  0.9674   |  5.0      |  0.999    |  25.0     |  43.32    |
|  8        |  0.9683   |  5.0      |  0.999    |  25.0     |  86.67    |
|  9        |  0.9674   |  15.0     |  0.1005   |  2.0      |  81.75    |
|  10       |  0.9666   |  9.448    |  0.7646   |  12.99    |  91.82    |
|  11       |  0.9714   |  5.794    |  0.6711   |  5.405    |  87.99    |
|  12       |  0.968    |  10.41    | 

## 随机森林回归


In [37]:
def rf_cv(n_estimators, min_samples_split, max_features, max_depth):
    rfr=RandomForestRegressor(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=min(max_features, 0.999), # float
            max_depth=int(max_depth),
            random_state=2)
    rfr.fit(train_xt,train_yt)
    pre=rfr.predict(train_xt)    
    val=r2_score(train_yt,pre)
    #val=rfr.score(train_xt,train_yt)
    return val
rf_bo = BayesianOptimization(
        rf_cv,
        {'n_estimators': (10, 250),
        'min_samples_split': (2, 25),
        'max_features': (0.1, 0.999),
        'max_depth': (5, 15)}
    )
rf_bo.maximize()

|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        |  0.9989   |  10.81    |  0.9219   |  24.45    |  74.73    |
|  2        |  0.9924   |  7.403    |  0.2774   |  17.88    |  90.54    |
|  3        |  0.9974   |  10.79    |  0.3385   |  8.391    |  14.6     |
|  4        |  0.9989   |  12.46    |  0.5617   |  12.59    |  74.63    |
|  5        |  0.9969   |  9.06     |  0.2313   |  23.6     |  190.2    |
|  6        |  0.9995   |  13.54    |  0.6773   |  18.5     |  70.91    |
|  7        |  0.9487   |  5.09     |  0.1      |  16.11    |  60.6     |
|  8        |  0.9893   |  5.0      |  0.999    |  19.7     |  10.0     |
|  9        |  0.9972   |  12.08    |  0.4041   |  18.93    |  71.68    |
|  10       |  1.0      |  15.0     |  0.999    |  2.0      |  86.2     |
|  11       |  1.0      |  15.0     |  0.999    |  2.0      |  103.5    |
|  12       |  1.0      |  14.99    | 